# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))   

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
#            print(line)
            full_data_rows_list.append(line) 
#            print("NEW")
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], Decimal(row[6]), row[7], row[8], row[12], row[13], row[16]))
        #print(row[6])        

df=pd.read_csv('event_datafile_new.csv')
#print(df[:1])
#print(full_data_rows_list[0])

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = {'class': 'SimpleStrategy','replication_factor':1}""")

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4 

query="CREATE TABLE IF NOT EXISTS music_lib"
query = query+"(artist text, fname text, gender text,item_session int,lname text,length decimal, level text,location varchar, session_id int,song text, user_id int,PRIMARY KEY(session_id, item_session))"

session.execute(query)

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_lib(artist,fname,gender,item_session,lname,length,level,location,session_id,song,user_id)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], Decimal(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))
        

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song,length FROM music_lib WHERE session_id = 338 AND item_session = 4"
rows=session.execute(query)

for row in rows:
    print("Artist:",row.artist,"Song:",row.song,"Duration:",row.length)

Artist: Faithless Song: Music Matters (Mark Knight Dub) Duration: 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [15]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query="""CREATE TABLE IF NOT EXISTS music_lib1 
       (artist text, 
        fname text, 
        gender text,
        item_session int,
        lname text,
        length decimal, 
        level text,
        location varchar, 
        session_id int,
        song text, 
        user_id int,
        PRIMARY KEY(session_id, user_id, item_session))""" 
        
session.execute(query)


# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_lib1(artist,fname,gender,item_session,lname,length,level,location,session_id,song,user_id)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], Decimal(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))


query = "SELECT artist, song,fname, lname, item_session FROM music_lib1 WHERE session_id = 182 AND user_id = 10"
rows=session.execute(query)

for row in rows:
    print("Artist is {} | Song is {} | User is {} | Item In Session is {}".format(row.artist,row.song,row.fname+' '+row.lname,row.item_session))

                    

Artist is Down To The Bone | Song is Keep On Keepin' On | User is Sylvie Cruz | Item In Session is 0
Artist is Three Drives | Song is Greece 2000 | User is Sylvie Cruz | Item In Session is 1
Artist is Sebastien Tellier | Song is Kilometer | User is Sylvie Cruz | Item In Session is 2
Artist is Lonnie Gordon | Song is Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | User is Sylvie Cruz | Item In Session is 3


In [19]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query="""CREATE TABLE IF NOT EXISTS music_lib2 
       (artist text, 
        fname text, 
        gender text,
        item_session int,
        lname text,
        length decimal, 
        level text,
        location varchar, 
        session_id int,
        song text, 
        user_id int,
        PRIMARY KEY(song,session_id,item_session))""" 
        
session.execute(query)


# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_lib2(artist,fname,gender,item_session,lname,length,level,location,session_id,song,user_id)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], Decimal(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))


query = "SELECT fname,lname FROM music_lib2 WHERE song = 'All Hands Against His Own'"
rows=session.execute(query)

print("List of every user who listened to the song 'All Hands Against His Own'")
for row in rows:
    print("User Name is: {}".format(row.fname+' '+row.lname))

                    

List of every user who listened to the song 'All Hands Against His Own'
User Name is: Sara Johnson
User Name is: Jacqueline Lynch
User Name is: Tegan Levine


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [21]:
query = "drop table music_lib"
rows = session.execute(query)
query = "drop table music_lib1"
rows = session.execute(query)
query = "drop table music_lib2"
rows = session.execute(query)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()